In [1]:
# imports
import scipy.io
import numpy as np               # for arrays
from numpy import linalg as LA   # for eigenvalues
import matplotlib                # for plots
import time                      # for time measurements
from PIL import Image            # for showing images
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sympy
from functions import *

In [2]:
def show_img(img):
    temp = img.copy()
    temp.resize((46,56))
    im = Image.fromarray(temp.T)
    im.show()

Get training/test splits

In [3]:
# load data
mat = scipy.io.loadmat('face.mat')
raw_data = mat['X']

raw_data = np.transpose(raw_data)
N,D = raw_data.shape
C = 52 # number of classes in dataset
train_size = int(N * 0.8)
test_size = int(N * 0.2)

pca_training_data = np.empty([int(520*0.8), 2576])
pca_testing_data = np.empty([int(520*0.2), 2576])
lda_training_data = []
lda_testing_data = []

# create training and test data
for x in range(52):
    # 8/2 ratio for training and testing datasets
    lda_training_data.append(raw_data[x*10:x*10+8].copy())
    lda_testing_data.append(raw_data[x*10+8:(x+1)*10].copy())
    

lda_training_data = np.array(lda_training_data)
lda_testing_data = np.array(lda_testing_data)
# pca_training_data = lda_training_data.reshape(train_size, D)
# pca_testing_data = lda_testing_data.reshape(test_size,D)


In [4]:
# load data
mat = scipy.io.loadmat('face.mat')
raw_data = mat['X']

D,N = raw_data.shape

raw_data = np.transpose(raw_data)

training_data = np.empty([int(520*0.8), 2576])
testing_data = np.empty([int(520*0.2), 2576])

# create training and test data
for x in range(52):
    training_data[x*8:(x+1)*8] = raw_data[x*10:x*10+8]
    testing_data[x*2:(x+1)*2] = raw_data[x*10+8:(x+1)*10]

raw_data = np.transpose(raw_data)
pca_training_data = np.transpose(training_data).T
pca_testing_data = np.transpose(testing_data).T



We need to make sure that the generalised eigenvalue problem that we encounter when doing LDA is solvable by making sure that the within-class scatter matrix is non-singular.
We do this by first reducing the dimension of the data via low dim PCA to an M <= N - c.

In [5]:
def get_Wpca(data, out_dim,random, M0, M1):
    # low-dim PCA
    S = data.dot(data.T)
    w, v = LA.eigh(S)
    u = data.T.dot(v)
    u = u.T # the eigenvectors aren't normalised after this
    u /= LA.norm(u, ord=2, axis=0)
    
    # sort wrt abs(eigenvalue)
    id = np.argsort(np.abs(w))[::-1]
    w = w[id]
    u = u[id]
    if random == True:
        origin = u[:M0].copy()
        remaining = u[M0:].copy()
        random_features = remaining[np.random.choice(remaining.shape[0], M1, replace=False), :]
        new_u = np.concatenate((origin,random_features), axis=0)
        print(new_u.T)
        return new_u.T
        
        
    else:
        print(u[0:out_dim].T.shape)
        return u[0:out_dim].T
    

In [6]:
# yo = get_Wpca(pca_training_data, 400, True, 50, 50)

We want a projection that maximises the ratio between the between-class scatter matrix and the within class scatter matrix.
The projection W turns out to be the solutions to the generalised eigen value problem. (Found via solving the langrangian. Slide 10-11)

In [7]:
def get_Wlda(Sb_data, Sw_data, out_dim):
    #code that gets value
    lda_evals, lda_evecs = LA.eig(LA.inv(Sw_data).dot(Sb_data))
    # print("magnitud of eigenvecs: ", LA.norm(lda_evecs.T[0], ord=2, axis=0))
    lda_evecs = lda_evecs.T
#     print("lda_evals: ", lda_evals)
    
    # sort wrt abs(eigenvalue)
    id = np.argsort(np.abs(lda_evals))[::-1]
    lda_evals = lda_evals[id]
    lda_evecs = lda_evecs[id]
    return lda_evecs[0:out_dim].T

In [8]:
def get_Wopt(pca_data, lda_data, Mpca, Mlda, random=False, M0=40, M1=10, committee_size=2):
    # random is true, returns tensor of random Wopts
    mean_all_data = pca_data.T.mean(axis=1).T

    # between class scatter (scalar)
    mean_class_data = lda_data.mean(axis=1)
    diff_class_mean = mean_class_data - mean_all_data
    Sb = np.dot(diff_class_mean.T, diff_class_mean)
    Sb *= np.array([8])

    # within class scatter (scalar)
    diff_class_data = lda_data - mean_class_data.reshape(52,1,-1)
    Sw = np.zeros((2576, 2576));
    for x in diff_class_data:
        Sw += np.dot(x.T,x)
    #use
    new_ensemble = np.empty([committee_size, 2576, M0 + M1])
    for x in range(committee_size):
        print(x)
        Wpca = get_Wpca(pca_data, Mpca, True, M0, M1)
        reduced_Sb = Wpca.T.dot(Sb).dot(Wpca) # symmetric matrix hopefully
        reduced_Sw = Wpca.T.dot(Sw).dot(Wpca)
        Wlda = get_Wlda(reduced_Sb, reduced_Sw, Mlda)

        Wopt = Wlda.T.dot(Wpca.T).T
        Wopt /= LA.norm(Wopt, ord=2, axis=1, keepdims=True)
        new_ensemble[x] = Wopt
        
#     print("Wopt: ", Wopt.shape, " Wlda.T: ", Wlda.T.shape, " Wpca.T ", Wpca.T.shape)
    
    return new_ensemble

We can now express each data point as product of a weight vector with Wopt. This weight vector can be used to classify each data point using nearest neighbour

In [13]:
def get_PCALDA_accuracy(Mpca, Mlda,random=False, M0=46, M1=1, committee_size=10):
    
    W = get_Wopt(pca_training_data, lda_training_data, Mpca, Mlda, random, M0, M1, committee_size)
#     print("reached here")
    
    training_face_weights = np.matmul(np.transpose(W,(0,2,1)),pca_training_data.T)
    testing_face_weights = np.matmul(np.transpose(W,(0,2,1)),pca_testing_data.T)
#     print("training_face_weights.shape", training_face_weights.shape)
#     print("tresting_face_weights.shape", testing_face_weights.shape)
    
    result = class_rate_tensor(np.transpose(training_face_weights.real,(0,2,1)), np.transpose(testing_face_weights.real, (0,2,1)))
    acc = 100*np.sum(result, axis=-1)/len(result)
    print("Mpca: ", Mpca, " Mlda: ", Mlda, " Classification rate: ", acc, "%.")
    return acc


In [10]:
# percent = get_PCALDA_accuracy(46,12)
print(percent)

NameError: name 'percent' is not defined

Now lets test the accuracy of the classifier as we vary Mpca from 1 to 416 and keeping LDA at 416. 
When the dimensions D is larger than the number of data points, the within-class covariance matrix becomes singular and so cannot be inverted. Therefore, PCA is used on the data before LDA in order to reduce the dimensions of the data and make Sw non-singular.

Another reason is that having an Mpca too high is known to make the model overfit. This is because when the dimension of data is much larger than the number of data points, finding a linear projection into a lower dimension that separates the classes completely becomes very easy and the projection found often does not generalise well. In this light, PCA can be seen as a regularisation technique.

https://stats.stackexchange.com/questions/106121/does-it-make-sense-to-combine-pca-and-lda

For this dataset, the best regularisation is Mpca = 46

In [ ]:
# results = []
# for i in range(1, 417):
#     results.append(get_PCALDA_accuracy(i, 417))


In [ ]:
results = np.loadtxt("VaryingPca.txt")
plot_data("Nearest Neigbhour Classification accuracy against Mpca", "Mpca", "Classification accuracy %", results)
# plt.savefig("Vary_Mpca_NN.png")
# np.savetxt("VaryingPca.txt", results)
# this graph shows that the test accuracy initially increases as Mpca increases as the model is given more 
# information to work with. Maximum accuracy is reached when Mpca is at 46. When Mpca increases above 46, 
# the model overfits and test accuracy decreases. This problem can be solved by using more data samples.


In [ ]:
# ldaresults = []
# for i in range(1, 417):
#     ldaresults.append(get_PCALDA_accuracy(47, i))


In [ ]:
ldaresults = np.loadtxt("VaryingLda.txt")
ldaresults2 = ldaresults[0:47]
plot_data("Nearest Neighbour Classification accuracy against Mlda 2","Mlda","Classification accuracy", ldaresults2)
# np.savetxt("VaryingLda.txt", ldaresults)
# this graph shows that only the first 50 largest eigenvalues from LDA give discriminate between the different classes
# the smallest eigenvalues are too small or zero

Ensembles aim to create stronger classifiers by fusing the predictions of multiple weaker classifiers in the hope that their individual strengths cover different areas of the input space or that the weak experts reinforce each others decisions. The weak classifiers in an ensemble can be created by either randomly sampling in feature space or by training on random subsets of the data set. There output is then fused by a fusion rule, the type of fusion rule chosen affects that overall performance of the model

First do pca, then do LDA on a random subset of those principal components
get their individual classifications 
get their combined classification using some fusion rule - READ UP
start decimating models randomly, down to 1 model to see how the number of models affects output score

Get random samples of data sets create samples randomly and then classify each data set in series
keep their models and run them individually on the whole dataset
combine their outputs using some fusion rule
decimate models randomly like above

try different fusion rules for each different model number

the effect of different fusion rules on a committee machines composed of varying number of weaker machines

Vary randomness parametre





In [14]:
ensemble_results = get_PCALDA_accuracy(416, 416, random=True, committee_size=20)

0
[[ 9.28306007e-01 -9.00656771e-02  4.37697997e-02 ...  2.41188421e-04
  -8.05277781e-03  8.32465982e-03]
 [ 9.29676296e-01 -9.67442415e-02  5.06287832e-02 ... -1.55347660e-02
  -1.09861577e-02  1.10953665e-03]
 [ 9.28088228e-01 -1.02934185e-01  5.98288545e-02 ... -2.60653446e-02
  -2.25315897e-02 -6.06860182e-03]
 ...
 [ 8.42166732e-01  3.09011440e-01  1.90171066e-01 ... -3.49012074e-02
   3.68743710e-03  3.01558984e-03]
 [ 8.36926421e-01  3.09991095e-01  1.88881687e-01 ... -4.97899055e-02
   2.50082387e-03 -4.66022454e-03]
 [ 8.30329750e-01  3.18890266e-01  1.83496822e-01 ... -4.80590157e-02
  -7.60365360e-03 -1.14630536e-02]]
1
[[ 9.28306007e-01 -9.00656771e-02  4.37697997e-02 ...  2.41188421e-04
  -8.05277781e-03 -4.19176292e-03]
 [ 9.29676296e-01 -9.67442415e-02  5.06287832e-02 ... -1.55347660e-02
  -1.09861577e-02 -1.65961794e-04]
 [ 9.28088228e-01 -1.02934185e-01  5.98288545e-02 ... -2.60653446e-02
  -2.25315897e-02 -3.83589869e-03]
 ...
 [ 8.42166732e-01  3.09011440e-01  1.901

14
[[ 9.28306007e-01 -9.00656771e-02  4.37697997e-02 ...  2.41188421e-04
  -8.05277781e-03  8.46544345e-03]
 [ 9.29676296e-01 -9.67442415e-02  5.06287832e-02 ... -1.55347660e-02
  -1.09861577e-02 -6.20717538e-04]
 [ 9.28088228e-01 -1.02934185e-01  5.98288545e-02 ... -2.60653446e-02
  -2.25315897e-02  1.65969072e-03]
 ...
 [ 8.42166732e-01  3.09011440e-01  1.90171066e-01 ... -3.49012074e-02
   3.68743710e-03  2.19617053e-03]
 [ 8.36926421e-01  3.09991095e-01  1.88881687e-01 ... -4.97899055e-02
   2.50082387e-03  4.69724060e-03]
 [ 8.30329750e-01  3.18890266e-01  1.83496822e-01 ... -4.80590157e-02
  -7.60365360e-03  5.50959437e-04]]
15
[[ 9.28306007e-01 -9.00656771e-02  4.37697997e-02 ...  2.41188421e-04
  -8.05277781e-03 -3.11486013e-03]
 [ 9.29676296e-01 -9.67442415e-02  5.06287832e-02 ... -1.55347660e-02
  -1.09861577e-02  1.01439088e-03]
 [ 9.28088228e-01 -1.02934185e-01  5.98288545e-02 ... -2.60653446e-02
  -2.25315897e-02  1.03587973e-03]
 ...
 [ 8.42166732e-01  3.09011440e-01  1.9

In [ ]:
    W = get_Wopt(pca_training_data, lda_training_data, 416, 416, True, 47, 70, committee_size)
#     print("reached here")
    
    training_face_weights = np.matmul(np.transpose(W,(0,2,1)),pca_training_data.T)
    testing_face_weights = np.matmul(np.transpose(W,(0,2,1)),pca_testing_data.T)
#     print("training_face_weights.shape", training_face_weights.shape)
#     print("tresting_face_weights.shape", testing_face_weights.shape)
    
    result = class_rate_tensor(np.transpose(training_face_weights.real,(0,2,1)), np.transpose(testing_face_weights.real, (0,2,1)))
    acc = 100*np.sum(result, axis=-1)/len(result)
    print("Mpca: ", Mpca, " Mlda: ", Mlda, " Classification rate: ", acc, "%.")
    return acc